In [10]:
import csv
import codecs
import urllib.request
import sys
from urllib.parse import urlparse

In [28]:
# This is the core of our weather query URL
BaseURL = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/'

In [74]:
# Set up the location parameter for our query
QueryLocation = '&location=43.232212,-70.829685' 

# Set up the query type parameter for our query ('FORECAST' or 'HISTORY')
QueryType= 'HISTORY'

# Set up the key parameter for our query
QueryKey = '&key=3IG6UAJ4EUV8THM9KCEX13TMU'

# Set up the date parameters for our query. Used only for historical weather data requests
FromDateParam = '2021-3-6'
ToDateParam = '2021-3-8'

In [75]:
# Set up the specific parameters based on the type of query
if QueryType == 'FORECAST':
    print(' - Fetching forecast data')
    QueryTypeParams = 'forecast?&aggregateHours=24&unitGroup=us&shortColumnNames=false'
else:
    print(' - Fetching history for date: ', FromDateParam,'-',ToDateParam)

    # History requests require a date.  We use the same date for start and end since we only want to query a single date in this example
    QueryDate = '&startDateTime=' + FromDateParam + 'T00:00:00&endDateTime=' +ToDateParam + 'T00:00:00'
    QueryTypeParams = 'history?&aggregateHours=24&unitGroup=us&dayStartTime=0:0:00&dayEndTime=0:0:00' + QueryDate

 - Fetching history for date:  2021-3-6 - 2021-3-8


In [76]:
# Build the entire query
URL = BaseURL + QueryTypeParams + QueryLocation + QueryKey

print(' - Running query URL: ', URL)
print()

 - Running query URL:  https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?&aggregateHours=24&unitGroup=us&dayStartTime=0:0:00&dayEndTime=0:0:00&startDateTime=2021-3-6T00:00:00&endDateTime=2021-3-8T00:00:00&location=43.232212,-70.829685&key=3IG6UAJ4EUV8THM9KCEX13TMU



In [77]:
# Parse the results as CSV
CSVBytes = urllib.request.urlopen(URL)
CSVText = csv.reader(codecs.iterdecode(CSVBytes, 'utf-8'))

In [78]:
RowIndex = 0

# The first row contain the headers and the additional rows each contain the weather metrics for a single day
# To simply our code, we use the knowledge that column 0 contains the location and column 1 contains the date.  The data starts at column 4
for Row in CSVText:
    if RowIndex == 0:
        FirstRow = Row
    else:
        print('Weather in ', Row[0], ' on ', Row[1])

        ColIndex = 0
        for Col in Row:
            if ColIndex >= 4:
                print('   ', FirstRow[ColIndex], ' = ', Row[ColIndex])
            ColIndex += 1
    RowIndex += 1

Weather in  43.232212,-70.829685  on  03/06/2021
    Temperature  =  23.2
    Dew Point  =  3.7
    Relative Humidity  =  42.71
    Heat Index  =  
    Wind Speed  =  11.1
    Wind Gust  =  21.9
    Wind Direction  =  297.46
    Wind Chill  =  4.0
    Precipitation  =  0.0
    Precipitation Cover  =  0.0
    Snow Depth  =  3.86
    Visibility  =  9.9
    Cloud Cover  =  15.2
    Sea Level Pressure  =  1014.6
    Weather Type  =  
    Latitude  =  43.232212
    Longitude  =  -70.829685
    Resolved Address  =  43.232212,-70.829685
    Name  =  43.232212,-70.829685
    Info  =  
    Conditions  =  Clear
Weather in  43.232212,-70.829685  on  03/07/2021
    Temperature  =  24.7
    Dew Point  =  6.1
    Relative Humidity  =  45.47
    Heat Index  =  
    Wind Speed  =  10.3
    Wind Gust  =  23.0
    Wind Direction  =  298.0
    Wind Chill  =  6.8
    Precipitation  =  0.0
    Precipitation Cover  =  0.0
    Snow Depth  =  3.32
    Visibility  =  9.9
    Cloud Cover  =  2.8
    Sea Level P

In [80]:
# If there are no CSV rows then something fundamental went wrong
if RowIndex == 0:
    print('Sorry, but it appears that there was an error connecting to the weather server.')
    print('Please check your network connection and try again..')

# If there is only one CSV  row then we likely got an error from the server
if RowIndex == 1:
    print('Sorry, but it appears that there was an error retrieving the weather data.')
    print('Error: ', FirstRow)